# Data loading and feature extraction 

In [1]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn import *
import glob

p = '/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/'

train = glob.glob(p+'train/**/**')
test = glob.glob(p+'test/**/**')
subjects = pd.read_csv(p+'subjects.csv')
tasks = pd.read_csv(p+'tasks.csv')
sub = pd.read_csv(p+'sample_submission.csv')

tdcsfog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv')
defog_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/defog_metadata.csv')
# daily_metadata=pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/daily_metadata.csv')
tdcsfog_metadata['Module']='tdcsfog'
defog_metadata['Module']='defog'
# daily_metadata['Module']='daily'
metadata=pd.concat([tdcsfog_metadata,defog_metadata])
metadata

,Id,Subject,Visit,Test,Medication,Module
0,003f117e14,4dc2f8,3,2.0,on,tdcsfog
1,009ee11563,f62eec,4,2.0,on,tdcsfog
2,011322847a,231c3b,2,2.0,on,tdcsfog
3,01d0fe7266,231c3b,2,1.0,off,tdcsfog
4,024418ba39,fa8764,19,3.0,on,tdcsfog
...,...,...,...,...,...,...
132,f3a921edee,1a778d,1,NaN,off,defog
133,f40e8c6ebe,575c60,1,NaN,off,defog
134,f8ddbdd98d,107712,1,NaN,on,defog
135,f9efef91fb,5d9cae,2,NaN,off,defog


In [2]:
# https://www.kaggle.com/code/jazivxt/familiar-solvs
tasks['Duration'] = tasks['End'] - tasks['Begin']
tasks = pd.pivot_table(tasks, values=['Duration'], index=['Id'], columns=['Task'], aggfunc='sum', fill_value=0)
tasks.columns = [c[-1] for c in tasks.columns]
tasks = tasks.reset_index()
tasks['t_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(tasks[tasks.columns[1:]])

subjects = subjects.fillna(0).groupby('Subject').median()
subjects = subjects.reset_index()
# subjects.rename(columns={'Subject':'Id'}, inplace=True)
subjects['s_kmeans'] = cluster.KMeans(n_clusters=10, random_state=3).fit_predict(subjects[subjects.columns[1:]])
subjects=subjects.rename(columns={'Visit':'s_Visit','Age':'s_Age','YearsSinceDx':'s_YearsSinceDx','UPDRSIII_On':'s_UPDRSIII_On','UPDRSIII_Off':'s_UPDRSIII_Off','NFOGQ':'s_NFOGQ'})

display(tasks)
display(subjects)

,Id,4MW,4MW-C,Hotspot1,Hotspot1-C,Hotspot2,Hotspot2-C,MB1,MB10,MB11,...,MB9,Rest1,Rest2,TUG-C,TUG-DT,TUG-ST,Turning-C,Turning-DT,Turning-ST,t_kmeans
0,02ab235146,16.520,16.680,16.760,16.240,53.920,64.600,13.960,17.960,17.400,...,30.800,180.48,60.32,38.440,47.920,36.240,21.920,46.400,23.320,2
1,02ea782681,11.618,11.796,11.525,11.692,8.329,9.032,3.469,6.624,6.230,...,30.650,0.00,0.00,18.343,19.932,20.130,18.042,21.588,18.698,0
2,06414383cf,24.860,41.584,25.885,0.000,38.642,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,44.832,33.867,0.000,83.837,124.299,3
3,092b4c1819,13.664,0.000,15.409,0.000,34.834,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,30.166,23.347,0.000,45.025,39.351,7
4,0a900ed8a2,11.720,11.840,10.600,10.720,43.171,41.160,1.760,21.040,10.560,...,30.520,180.88,60.32,18.083,25.000,19.920,18.699,18.200,16.880,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,f3a921edee,16.722,0.000,16.383,0.000,76.200,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,29.043,29.820,0.000,136.840,76.960,3
133,f40e8c6ebe,12.867,0.000,27.906,0.000,152.333,0.000,0.000,0.000,0.000,...,0.000,0.00,0.00,0.000,36.805,28.232,0.000,98.579,69.889,3
134,f8ddbdd98d,23.787,0.000,32.556,0.000,38.557,0.000,6.347,10.356,10.655,...,33.289,0.00,0.00,0.000,44.132,54.993,0.000,100.433,72.152,3
135,f9efef91fb,16.351,17.552,16.155,15.166,67.059,51.715,0.000,0.000,0.000,...,0.000,0.00,0.00,23.483,23.541,22.137,36.898,38.632,30.831,7


,Subject,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,s_kmeans
0,00f674,1.5,63.0,27.0,37.0,39.5,25.0,9
1,02bc69,0.0,69.0,4.0,21.0,0.0,22.0,7
2,040587,1.5,75.0,26.0,49.5,72.0,22.5,6
3,056372,2.0,69.0,13.0,44.0,50.0,22.0,0
4,07285e,0.0,58.0,1.0,18.0,26.0,10.0,5
...,...,...,...,...,...,...,...,...
131,f686f0,0.0,61.0,7.0,44.0,0.0,24.0,7
132,f80507,1.0,57.0,2.0,12.0,0.0,0.0,1
133,fa8764,0.0,60.0,7.0,30.0,0.0,19.0,7
134,fba3a3,1.0,65.0,8.0,28.0,0.0,0.0,1


In [3]:
complex_featlist=['Visit','Test','Medication','s_Visit','s_Age','s_YearsSinceDx','s_UPDRSIII_On','s_UPDRSIII_Off','s_NFOGQ','s_kmeans']
metadata_complex=metadata.merge(subjects,how='left',on='Subject').copy()
metadata_complex['Medication']=metadata_complex['Medication'].factorize()[0]

display(metadata_complex)

,Id,Subject,Visit,Test,Medication,Module,s_Visit,s_Age,s_YearsSinceDx,s_UPDRSIII_On,s_UPDRSIII_Off,s_NFOGQ,s_kmeans
0,003f117e14,4dc2f8,3,2.0,0,tdcsfog,0.0,68.0,9.0,17.0,15.0,15.0,5
1,009ee11563,f62eec,4,2.0,0,tdcsfog,0.0,71.0,10.0,42.0,0.0,24.0,3
2,011322847a,231c3b,2,2.0,0,tdcsfog,0.0,67.0,12.0,27.0,28.0,19.0,5
3,01d0fe7266,231c3b,2,1.0,1,tdcsfog,0.0,67.0,12.0,27.0,28.0,19.0,5
4,024418ba39,fa8764,19,3.0,0,tdcsfog,0.0,60.0,7.0,30.0,0.0,19.0,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,f3a921edee,1a778d,1,NaN,1,defog,1.5,65.0,7.0,50.0,59.5,24.5,8
966,f40e8c6ebe,575c60,1,NaN,1,defog,1.0,28.0,4.0,54.0,50.0,25.0,8
967,f8ddbdd98d,107712,1,NaN,0,defog,1.0,82.0,11.0,38.0,42.0,21.0,0
968,f9efef91fb,5d9cae,2,NaN,1,defog,1.5,72.0,14.0,22.5,39.0,16.0,2


## Create a tsflex feature collection

In [4]:
def preprocess_defog(df):
        
        if 'Valid' in df.columns:
            
            df.loc[df.Valid == 1, ['StartHesitation', 'Walking', 'Turn']] = df.loc[df.Valid == 1, ['StartHesitation', 'Walking', 'Turn']]
            df.loc[df.Valid == 0, ['StartHesitation', 'Walking', 'Turn']] = 0
        
        # Calculate the acceleration amplitude of -5 and +5 rows
        #df['max_amp_V'] = df['AccV'].rolling(window=21, center=True).apply(lambda x: max(x) - min(x), raw=True)
        #df['max_amp_ML'] = df['AccML'].rolling(window=21, center=True).apply(lambda x: max(x) - min(x), raw=True)
        #df['max_amp_AP'] = df['AccAP'].rolling(window=21, center=True).apply(lambda x: max(x) - min(x), raw=True)

        # calculate features over a rolling window of size 100
        window_size = 1000
        df[['AccML_mean_3000','AccAP_mean_3000']] = df[['AccML','AccAP']].rolling(window=(1000)).mean()
        df['AccAP_mean_2000'] = df['AccAP'].rolling(window=(2000)).mean()
        
        # Calculate the rolling mean of the previous 5 rows of the accelerometer data column
        rolling_mean_V = df['AccV'].rolling(window=100).mean().shift()
        rolling_mean_ML = df['AccML'].rolling(window=100).mean().shift()
        rolling_mean_AP = df['AccAP'].rolling(window=100).mean().shift()

        # Subtract the rolling mean from the accelerometer data to center it around zero
        centered_data_V = df['AccV'] - rolling_mean_V
        centered_data_ML = df['AccML'] - rolling_mean_ML
        centered_data_AP = df['AccAP'] - rolling_mean_AP

        # Create new columns for the upper and lower parts of the vibrations
        df['Upper_V'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Lower_V'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Upper_ML'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Lower_ML'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Upper_AP'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Lower_AP'] = pd.Series([float('nan')]*len(df), dtype=float)

        # Iterate over the centered accelerometer data and assign values to the upper and lower columns
        for i in range(len(centered_data_V)):
            if centered_data_V[i] >= 0:
                df.at[i, 'Upper_V'] = centered_data_V[i]
            else:
                df.at[i, 'Lower_V'] = centered_data_V[i]
            if centered_data_ML[i] >= 0:
                df.at[i, 'Upper_ML'] = centered_data_ML[i]
            else:
                df.at[i, 'Lower_ML'] = centered_data_ML[i]
            if centered_data_AP[i] >= 0:
                df.at[i, 'Upper_AP'] = centered_data_AP[i]
            else:
                df.at[i, 'Lower_AP'] = centered_data_AP[i]

        # Interpolate the missing values in the Upper and Lower columns
        df['Upper_V'].interpolate(method='linear', inplace=True)
        df['Lower_V'].interpolate(method='linear', inplace=True)
        df['Upper_ML'].interpolate(method='linear', inplace=True)
        df['Lower_ML'].interpolate(method='linear', inplace=True)
        df['Upper_AP'].interpolate(method='linear', inplace=True)
        df['Lower_AP'].interpolate(method='linear', inplace=True)

        # Add the rolling mean back to the data to obtain the final values in the original scale
        df['Upper_V'] += rolling_mean_V
        df['Lower_V'] += rolling_mean_V
        df['Upper_ML'] += rolling_mean_ML
        df['Lower_ML'] += rolling_mean_ML
        df['Upper_AP'] += rolling_mean_AP
        df['Lower_AP'] += rolling_mean_AP

        # Calculate the differences
        df['diff_V'] = df['Upper_V']-df['Lower_V']
        df['diff_ML'] = df['Upper_ML']-df['Lower_ML']
        df['diff_AP'] = df['Upper_AP']-df['Lower_AP']
        #df_tdcsfog['diff_tot'] = df_tdcsfog['diff_V']+df_tdcsfog['diff_ML']+df_tdcsfog['diff_AP']

        # Calculate rolling sums
        #df['rolling_sum_V_2000'] = df['diff_V'].rolling(window=2000, min_periods=1).sum()
        #df['rolling_sum_ML_2000'] = df['diff_ML'].rolling(window=2000, min_periods=1).sum()
        #df['rolling_sum_AP_2000'] = df['diff_AP'].rolling(window=2000, min_periods=1).sum()
        #df['rolling_sum_V_1000'] = df['diff_V'].rolling(window=window_size, min_periods=1).sum()
        #df['rolling_sum_ML_1000'] = df['diff_ML'].rolling(window=window_size, min_periods=1).sum()
        #df['rolling_sum_AP_1000'] = df['diff_AP'].rolling(window=window_size, min_periods=1).sum()

        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=100, min_periods=1).std()
        #rolling_std_ML = df['AccML'].rolling(window=100, min_periods=1).std()
        #rolling_std_AP = df['AccAP'].rolling(window=100, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        #df['rolling_avg_std_V_100'] = rolling_std_V.rolling(window=100, min_periods=1).mean()
        #df['rolling_avg_std_ML_100'] = rolling_std_ML.rolling(window=100, min_periods=1).mean()
        #df['rolling_avg_std_AP_100'] = rolling_std_AP.rolling(window=100, min_periods=1).mean()
        
        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=500, min_periods=1).std()
        #rolling_std_ML = df['AccML'].rolling(window=500, min_periods=1).std()
        rolling_std_AP = df['AccAP'].rolling(window=500, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        #df['rolling_avg_std_V_500'] = rolling_std_V.rolling(window=500, min_periods=1).mean()
        #df['rolling_avg_std_ML_500'] = rolling_std_ML.rolling(window=500, min_periods=1).mean()
        df['rolling_avg_std_AP_500'] = rolling_std_AP.rolling(window=500, min_periods=1).mean()
        
        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=1000, min_periods=1).std()
        #rolling_std_ML = df['AccML'].rolling(window=1000, min_periods=1).std()
        #rolling_std_AP = df['AccAP'].rolling(window=1000, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        #df['rolling_avg_std_V_1000'] = rolling_std_V.rolling(window=1000, min_periods=1).mean()
        #df['rolling_avg_std_ML_1000'] = rolling_std_ML.rolling(window=1000, min_periods=1).mean()
        #df['rolling_avg_std_AP_1000'] = rolling_std_AP.rolling(window=1000, min_periods=1).mean()
        
        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=2000, min_periods=1).std()
        #rolling_std_ML = df['AccML'].rolling(window=2000, min_periods=1).std()
        rolling_std_AP = df['AccAP'].rolling(window=2000, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        #df['rolling_avg_std_V_2000'] = rolling_std_V.rolling(window=2000, min_periods=1).mean()
        #df['rolling_avg_std_ML_2000'] = rolling_std_ML.rolling(window=2000, min_periods=1).mean()
        df['rolling_avg_std_AP_2000'] = rolling_std_AP.rolling(window=2000, min_periods=1).mean()
        
        df['Time_frac']=(df.index/df.index.max()).values #currently the index of data is actually "Time"
        
        df = df.drop(['Upper_V','Lower_V','Upper_ML','Lower_ML','Upper_AP','Lower_AP',
                      'diff_V','diff_ML','diff_AP','AccML','AccV','AccAP'], axis=1)
        
        # df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
        # df = pd.merge(df, subjects[['Id','s_kmeans']], how='left', on='Id').fillna(-1)
        df = pd.merge(df, metadata_complex[['Id','Subject']], how='left', on='Id').fillna(-1)
        
        # df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        # df = df.merge(df_feats, how="left", left_index=True, right_index=True)
        df.fillna(method="ffill", inplace=True)
        
        return df

In [5]:
def preprocess_tdcsfog(df):
        
        if 'Valid' in df.columns:
            
            df.loc[df.Valid == 1, ['StartHesitation', 'Walking', 'Turn']] = df.loc[df.Valid == 1, ['StartHesitation', 'Walking', 'Turn']]
            df.loc[df.Valid == 0, ['StartHesitation', 'Walking', 'Turn']] = 0
        
        # Calculate the acceleration amplitude of -5 and +5 rows
        # df['max_amp_V'] = df['AccV'].rolling(window=21, center=True).apply(lambda x: max(x) - min(x), raw=True)
        # df['max_amp_ML'] = df['AccML'].rolling(window=21, center=True).apply(lambda x: max(x) - min(x), raw=True)
        # df['max_amp_AP'] = df['AccAP'].rolling(window=21, center=True).apply(lambda x: max(x) - min(x), raw=True)

        # calculate features over a rolling window of size 100
        window_size = 1000
        df[['AccV_mean_5000','AccAP_mean_500']] = df[['AccV','AccAP']].rolling(window=(500)).mean()
        df['AccAP_mean_1000'] = df['AccAP'].rolling(window=(1000)).mean()
        df['AccV_mean_2000'] = df['AccV'].rolling(window=(2000)).mean()
        df['AccV_mean_3000'] = df['AccV'].rolling(window=(3000)).mean()
        
        # Calculate the rolling mean of the previous 5 rows of the accelerometer data column
        rolling_mean_V = df['AccV'].rolling(window=100).mean().shift()
        rolling_mean_ML = df['AccML'].rolling(window=100).mean().shift()
        rolling_mean_AP = df['AccAP'].rolling(window=100).mean().shift()

        # Subtract the rolling mean from the accelerometer data to center it around zero
        centered_data_V = df['AccV'] - rolling_mean_V
        centered_data_ML = df['AccML'] - rolling_mean_ML
        centered_data_AP = df['AccAP'] - rolling_mean_AP

        # Create new columns for the upper and lower parts of the vibrations
        df['Upper_V'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Lower_V'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Upper_ML'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Lower_ML'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Upper_AP'] = pd.Series([float('nan')]*len(df), dtype=float)
        df['Lower_AP'] = pd.Series([float('nan')]*len(df), dtype=float)

        # Iterate over the centered accelerometer data and assign values to the upper and lower columns
        for i in range(len(centered_data_V)):
            if centered_data_V[i] >= 0:
                df.at[i, 'Upper_V'] = centered_data_V[i]
            else:
                df.at[i, 'Lower_V'] = centered_data_V[i]
            if centered_data_ML[i] >= 0:
                df.at[i, 'Upper_ML'] = centered_data_ML[i]
            else:
                df.at[i, 'Lower_ML'] = centered_data_ML[i]
            if centered_data_AP[i] >= 0:
                df.at[i, 'Upper_AP'] = centered_data_AP[i]
            else:
                df.at[i, 'Lower_AP'] = centered_data_AP[i]

        # Interpolate the missing values in the Upper and Lower columns
        df['Upper_V'].interpolate(method='linear', inplace=True)
        df['Lower_V'].interpolate(method='linear', inplace=True)
        df['Upper_ML'].interpolate(method='linear', inplace=True)
        df['Lower_ML'].interpolate(method='linear', inplace=True)
        df['Upper_AP'].interpolate(method='linear', inplace=True)
        df['Lower_AP'].interpolate(method='linear', inplace=True)

        # Add the rolling mean back to the data to obtain the final values in the original scale
        df['Upper_V'] += rolling_mean_V
        df['Lower_V'] += rolling_mean_V
        df['Upper_ML'] += rolling_mean_ML
        df['Lower_ML'] += rolling_mean_ML
        df['Upper_AP'] += rolling_mean_AP
        df['Lower_AP'] += rolling_mean_AP

        # Calculate the differences
        df['diff_V'] = df['Upper_V']-df['Lower_V']
        df['diff_ML'] = df['Upper_ML']-df['Lower_ML']
        df['diff_AP'] = df['Upper_AP']-df['Lower_AP']
        #df_tdcsfog['diff_tot'] = df_tdcsfog['diff_V']+df_tdcsfog['diff_ML']+df_tdcsfog['diff_AP']

        # Calculate rolling sums
        df['rolling_sum_V_2000'] = df['diff_V'].rolling(window=2000, min_periods=1).sum()
        #df['rolling_sum_ML_2000'] = df['diff_ML'].rolling(window=2000, min_periods=1).sum()
        #df['rolling_sum_AP_2000'] = df['diff_AP'].rolling(window=2000, min_periods=1).sum()
        #df['rolling_sum_V_1000'] = df['diff_V'].rolling(window=window_size, min_periods=1).sum()
        #df['rolling_sum_ML_1000'] = df['diff_ML'].rolling(window=window_size, min_periods=1).sum()
        #df['rolling_sum_AP_1000'] = df['diff_AP'].rolling(window=window_size, min_periods=1).sum()

        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=100, min_periods=1).std()
        #rolling_std_ML = df['AccML'].rolling(window=100, min_periods=1).std()
        #rolling_std_AP = df['AccAP'].rolling(window=100, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        # df['rolling_avg_std_V_100'] = rolling_std_V.rolling(window=100, min_periods=1).mean()
        #df['rolling_avg_std_ML_100'] = rolling_std_ML.rolling(window=100, min_periods=1).mean()
        #df['rolling_avg_std_AP_100'] = rolling_std_AP.rolling(window=100, min_periods=1).mean()
        
        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=500, min_periods=1).std()
        #rolling_std_ML = df['AccML'].rolling(window=500, min_periods=1).std()
        #rolling_std_AP = df['AccAP'].rolling(window=500, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        #df['rolling_avg_std_V_500'] = rolling_std_V.rolling(window=500, min_periods=1).mean()
        #df['rolling_avg_std_ML_500'] = rolling_std_ML.rolling(window=500, min_periods=1).mean()
        #df['rolling_avg_std_AP_500'] = rolling_std_AP.rolling(window=500, min_periods=1).mean()
        
        # Compute rolling standard deviation with a window size of 100
        #rolling_std_V = df['AccV'].rolling(window=1000, min_periods=1).std()
        rolling_std_ML = df['AccML'].rolling(window=1000, min_periods=1).std()
        rolling_std_AP = df['AccAP'].rolling(window=1000, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        #df['rolling_avg_std_V_1000'] = rolling_std_V.rolling(window=1000, min_periods=1).mean()
        df['rolling_avg_std_ML_1000'] = rolling_std_ML.rolling(window=1000, min_periods=1).mean()
        df['rolling_avg_std_AP_1000'] = rolling_std_AP.rolling(window=1000, min_periods=1).mean()
        
        # Compute rolling standard deviation with a window size of 100
        rolling_std_V = df['AccV'].rolling(window=2000, min_periods=1).std()
        rolling_std_ML = df['AccML'].rolling(window=2000, min_periods=1).std()
        rolling_std_AP = df['AccAP'].rolling(window=2000, min_periods=1).std()

        # Compute rolling average of standard deviation with a window size of 100
        df['rolling_avg_std_V_2000'] = rolling_std_V.rolling(window=2000, min_periods=1).mean()
        df['rolling_avg_std_ML_2000'] = rolling_std_ML.rolling(window=2000, min_periods=1).mean()
        df['rolling_avg_std_AP_2000'] = rolling_std_AP.rolling(window=2000, min_periods=1).mean()
        
        df['Time_frac']=(df.index/df.index.max()).values #currently the index of data is actually "Time"
        
        df = df.drop(['Upper_V','Lower_V','Upper_ML','Lower_ML','Upper_AP','Lower_AP',
                      'diff_V','diff_ML','diff_AP','AccML','AccV','AccAP'], axis=1)
        
        # df = pd.merge(df, tasks[['Id','t_kmeans']], how='left', on='Id').fillna(-1)
        # df = pd.merge(df, subjects[['Id','s_kmeans']], how='left', on='Id').fillna(-1)
        df = pd.merge(df, metadata_complex[['Id','Subject']], how='left', on='Id').fillna(-1)
        
        # df_feats = fc.calculate(df, return_df=True, include_final_window=True, approve_sparsity=True, window_idx="begin").astype(np.float32)
        # df = df.merge(df_feats, how="left", left_index=True, right_index=True)
        df.fillna(method="ffill", inplace=True)
        
        return df

## Extract the features (with Time_frac feature)

In [6]:
import matplotlib.pyplot as plt
import pathlib
from sklearn.preprocessing import MinMaxScaler

def reader(f):

    try:
        df = pd.read_csv(f, index_col="Time", usecols=['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn' , 'Walking'])
        
        df['Id'] = f.split('/')[-1].split('.')[0]
        df['Module'] = pathlib.Path(f).parts[-2]
        
        if 'defog' in str(f):
            df = preprocess_defog(df)
        else:
            df = preprocess_tdcsfog(df)
        
        # remove the columns from their original positions
        start_hesitation = df.pop('StartHesitation')
        turn = df.pop('Turn')
        walking = df.pop('Walking')
        id_ = df.pop('Id')
        module = df.pop('Module')
        subject = df.pop('Subject')

        # add the columns at the end
        df.insert(len(df.columns), 'StartHesitation', start_hesitation)
        df.insert(len(df.columns), 'Turn', turn)
        df.insert(len(df.columns), 'Walking', walking)
        df.insert(0, 'Subject', subject)
        df.insert(0, 'Module', module)
        df.insert(0, 'Id', id_)
        
        # assume df is your DataFrame
        cols_to_scale = df.columns[3:-3]
        scaler = MinMaxScaler(feature_range=(-1, 1))

        df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
        
        # plot df
        # df[df.columns[3:]].plot(kind='line', figsize=(8, 6))
        # plt.show()
        
        return df
    
    except Exception as e:
        # code to handle the exception and print the error message
        print(f"An exception occurred: {e}")

train_defog = pd.concat([reader(f) for f in tqdm(train) if 'defog' in str(f)]).fillna(0); print('train_defog shape',train_defog.shape)
train_tdcsfog = pd.concat([reader(f) for f in tqdm(train) if 'tdcsfog' in str(f)]).fillna(0); print('train_tdcsfog shape',train_tdcsfog.shape)

cols_defog = [c for c in train_defog.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]
cols_tdcsfog = [c for c in train_tdcsfog.columns if c not in ['Id','Subject','Module', 'Time', 'StartHesitation', 'Turn' , 'Walking', 'Valid', 'Task','Event']]

pcols = ['StartHesitation', 'Turn' , 'Walking']
scols = ['Id', 'StartHesitation', 'Turn' , 'Walking']
train_defog=train_defog.reset_index(drop=True)
train_tdcsfog=train_tdcsfog.reset_index(drop=True)

  0%|          | 0/970 [00:00<?, ?it/s]

train_defog shape (13525702, 12)


  0%|          | 0/970 [00:00<?, ?it/s]

train_tdcsfog shape (7062672, 18)


# Build the model and submit

## Train the model

[0.3672348518792581, 0.31159614808283764, 0.07163485094899517, 0.28650052690052524, 0.07967279232646186]

In [7]:
import xgboost as xgb
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import GroupKFold, RandomizedSearchCV
from sklearn.metrics import average_precision_score, make_scorer
from scipy.stats import uniform, randint
from sklearn.base import clone

# Define the parameter grid for RandomizedSearchCV
param_grid = {
    'estimator__colsample_bytree': uniform(0.1, 0.9),
    'estimator__learning_rate': uniform(0.01, 0.3),
    'estimator__max_depth': randint(3, 10),
    'estimator__min_child_weight': uniform(1, 10),
    'estimator__n_estimators': randint(100, 500),
    'estimator__subsample': uniform(0.5, 0.5)
}

def custom_average_precision(y_true, y_pred):
    score = average_precision_score(y_true, y_pred)
    return 'average_precision', score, True

class LGBMMultiOutputRegressor(MultiOutputRegressor):
    def fit(self, X, y, eval_set=None, **fit_params):
        self.estimators_ = [clone(self.estimator) for _ in range(y.shape[1])]

        for i, estimator in enumerate(self.estimators_):
            if eval_set:
                fit_params['eval_set'] = [(eval_set[0], eval_set[1][:, i])]
            estimator.fit(X, y[:, i], **fit_params)

        return self

N_FOLDS = 5

# Training defog
kfold = GroupKFold(N_FOLDS)
group_var = train_defog.Subject
groups = kfold.split(train_defog, groups=group_var)
regs_defog = []
cvs_defog = []

for fold, (tr_idx, te_idx) in enumerate(tqdm(groups, total=N_FOLDS, desc="Folds")):
    tr_idx = pd.Series(tr_idx).sample(n=2000000, random_state=42).values

    # Create a base LGBMRegressor
    base_regressor = lgb.LGBMRegressor()

    # Wrap the base regressor with the MultiOutputRegressor
    multioutput_regressor = LGBMMultiOutputRegressor(base_regressor)

    x_tr, y_tr = train_defog.loc[tr_idx, cols_defog].to_numpy(), train_defog.loc[tr_idx, pcols].to_numpy()
    x_te, y_te = train_defog.loc[te_idx, cols_defog].to_numpy(), train_defog.loc[te_idx, pcols].to_numpy()

    # Perform RandomizedSearchCV
    randomized_search = RandomizedSearchCV(
        multioutput_regressor,
        param_distributions=param_grid,
        n_iter=10,  # Number of parameter combinations to try
        scoring=make_scorer(average_precision_score, greater_is_better=True),
        cv=3,
        refit=False
    )

    randomized_search.fit(x_tr, y_tr)
    
    # Get the best parameters found by RandomizedSearchCV
    best_params_ = randomized_search.best_params_

    # Print the best parameters
    print("Best Parameters:")
    for param, value in best_params_.items():
        print(f"{param}: {value}")
    
    # Get the best parameters found by RandomizedSearchCV
    best_params_ = randomized_search.best_params_
    best_params_ = {kk: v for k, v in best_params_.items() for kk in k.split('__')}; del best_params_['estimator']

    # Create a new regressor with the best parameters
    best_regressor = LGBMMultiOutputRegressor(lgb.LGBMRegressor(**best_params_))

    best_regressor.fit(
        x_tr, y_tr,
        eval_set=(x_te, y_te),
        eval_metric=custom_average_precision,
        early_stopping_rounds=25
    )
    
    regs_defog.append(best_regressor)
    cv = average_precision_score(y_te, best_regressor.predict(x_te).clip(0.0, 1.0))
    cvs_defog.append(cv)

print(cvs_defog)

# Training tdcsfog
kfold = GroupKFold(N_FOLDS)
group_var = train_tdcsfog.Subject
groups = kfold.split(train_tdcsfog, groups=group_var)
regs_tdcsfog = []
cvs_tdcsfog = []

for fold, (tr_idx, te_idx) in enumerate(tqdm(groups, total=N_FOLDS, desc="Folds")):
    tr_idx = pd.Series(tr_idx).sample(n=2000000, random_state=42).values

    # Create a base LGBMRegressor
    base_regressor = lgb.LGBMRegressor()

    # Wrap the base regressor with the MultiOutputRegressor
    multioutput_regressor = LGBMMultiOutputRegressor(base_regressor)

    x_tr, y_tr = train_tdcsfog.loc[tr_idx, cols_tdcsfog].to_numpy(), train_tdcsfog.loc[tr_idx, pcols].to_numpy()
    x_te, y_te = train_tdcsfog.loc[te_idx, cols_tdcsfog].to_numpy(), train_tdcsfog.loc[te_idx, pcols].to_numpy()

    # Perform RandomizedSearchCV
    randomized_search = RandomizedSearchCV(
        multioutput_regressor,
        param_distributions=param_grid,
        n_iter=10,  # Number of parameter combinations to try
        scoring=make_scorer(average_precision_score, greater_is_better=True),
        cv=3,
        refit=False
    )

    randomized_search.fit(x_tr, y_tr)
    
    # Get the best parameters found by RandomizedSearchCV
    best_params_ = randomized_search.best_params_

    # Print the best parameters
    print("Best Parameters:")
    for param, value in best_params_.items():
        print(f"{param}: {value}")
    
    # Get the best parameters found by RandomizedSearchCV
    best_params_ = randomized_search.best_params_
    best_params_ = {kk: v for k, v in best_params_.items() for kk in k.split('__')}; del best_params_['estimator']

    # Create a new regressor with the best parameters
    best_regressor = LGBMMultiOutputRegressor(lgb.LGBMRegressor(**best_params_))

    best_regressor.fit(
        x_tr, y_tr,
        eval_set=(x_te, y_te),
        eval_metric=custom_average_precision,
        early_stopping_rounds=25
    )
    
    regs_tdcsfog.append(best_regressor)
    cv = average_precision_score(y_te, best_regressor.predict(x_te).clip(0.0, 1.0))
    cvs_tdcsfog.append(cv)

print(cvs_tdcsfog)

Folds:   0%|          | 0/5 [00:00<?, ?it/s]

Best Parameters:
estimator__colsample_bytree: 0.6271392809711324
estimator__learning_rate: 0.1371264929813268
estimator__max_depth: 8
estimator__min_child_weight: 8.887711181232415
estimator__n_estimators: 369
estimator__subsample: 0.5108451171940764


/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 4.56324e-05	valid_0's average_precision: 4.53794e-05
[2]	valid_0's l2: 4.75357e-05	valid_0's average_precision: 3.68199e-05
[3]	valid_0's l2: 4.75986e-05	valid_0's average_precision: 3.96752e-05
[4]	valid_0's l2: 5.21668e-05	valid_0's average_precision: 4.19616e-05
[5]	valid_0's l2: 5.83824e-05	valid_0's average_precision: 9.06904e-05
[6]	valid_0's l2: 5.84362e-05	valid_0's average_precision: 5.26015e-05
[7]	valid_0's l2: 5.84435e-05	valid_0's average_precision: 5.12225e-05
[8]	valid_0's l2: 5.85191e-05	valid_0's average_precision: 5.09183e-05
[9]	valid_0's l2: 5.85327e-05	valid_0's average_precision: 5.47186e-05
[10]	valid_0's l2: 5.85792e-05	valid_0's average_precision: 5.46913e-05
[11]	valid_0's l2: 5.87067e-05	valid_0's average_precision: 5.4708e-05
[12]	valid_0's l2: 5.87267e-05	valid_0's average_precision: 7.71526e-05
[13]	valid_0's l2: 5.88926e-05	valid_0's average_precision: 7.66321e-05
[14]	valid_0's l2: 5.88926e-05	valid_0's average_precision: 7.6136e-05
[15

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


...

[1]	valid_0's l2: 0.00999834	valid_0's average_precision: 0.055253
[2]	valid_0's l2: 0.00998785	valid_0's average_precision: 0.0459479
[3]	valid_0's l2: 0.00973114	valid_0's average_precision: 0.0602193
[4]	valid_0's l2: 0.00963919	valid_0's average_precision: 0.0732464
[5]	valid_0's l2: 0.010325	valid_0's average_precision: 0.0579441
[6]	valid_0's l2: 0.0103003	valid_0's average_precision: 0.0644792
[7]	valid_0's l2: 0.0104562	valid_0's average_precision: 0.0648223
[8]	valid_0's l2: 0.010513	valid_0's average_precision: 0.0729178
[9]	valid_0's l2: 0.0105648	valid_0's average_precision: 0.0825811
[10]	valid_0's l2: 0.0107408	valid_0's average_precision: 0.0757953
[11]	valid_0's l2: 0.0108936	valid_0's average_precision: 0.0721112
[12]	valid_0's l2: 0.0110136	valid_0's average_precision: 0.0686242
[13]	valid_0's l2: 0.011179	valid_0's average_precision: 0.0676245
[14]	valid_0's l2: 0.0111916	valid_0's average_precision: 0.069112
[15]	valid_0's l2: 0.0112977	valid_0's average_precision: 

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.125699	valid_0's average_precision: 0.22866
[2]	valid_0's l2: 0.122553	valid_0's average_precision: 0.229639
[3]	valid_0's l2: 0.118287	valid_0's average_precision: 0.261508
[4]	valid_0's l2: 0.119261	valid_0's average_precision: 0.264519
[5]	valid_0's l2: 0.117575	valid_0's average_precision: 0.26581
[6]	valid_0's l2: 0.115614	valid_0's average_precision: 0.280618
[7]	valid_0's l2: 0.117645	valid_0's average_precision: 0.271712
[8]	valid_0's l2: 0.118193	valid_0's average_precision: 0.270613
[9]	valid_0's l2: 0.117298	valid_0's average_precision: 0.274804
[10]	valid_0's l2: 0.116639	valid_0's average_precision: 0.280637
[11]	valid_0's l2: 0.117159	valid_0's average_precision: 0.278486
[12]	valid_0's l2: 0.116135	valid_0's average_precision: 0.282155
[13]	valid_0's l2: 0.11659	valid_0's average_precision: 0.279574
[14]	valid_0's l2: 0.116946	valid_0's average_precision: 0.279181
[15]	valid_0's l2: 0.118379	valid_0's average_precision: 0.274076
[16]	valid_0's l2: 0.1

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's l2: 0.013416	valid_0's average_precision: 0.0189702
[2]	valid_0's l2: 0.0133667	valid_0's average_precision: 0.0203819
[3]	valid_0's l2: 0.0132742	valid_0's average_precision: 0.0232131
[4]	valid_0's l2: 0.0133194	valid_0's average_precision: 0.0224579
[5]	valid_0's l2: 0.0133161	valid_0's average_precision: 0.0230681
[6]	valid_0's l2: 0.0132815	valid_0's average_precision: 0.0237514
[7]	valid_0's l2: 0.0133079	valid_0's average_precision: 0.0238838
[8]	valid_0's l2: 0.0133088	valid_0's average_precision: 0.0242681
[9]	valid_0's l2: 0.0133518	valid_0's average_precision: 0.0246446
[10]	valid_0's l2: 0.0133686	valid_0's average_precision: 0.0245917
[11]	valid_0's l2: 0.0133708	valid_0's average_precision: 0.0241491
[12]	valid_0's l2: 0.0133958	valid_0's average_precision: 0.0231151
[13]	valid_0's l2: 0.0134296	valid_0's average_precision: 0.022317
[14]	valid_0's l2: 0.013432	valid_0's average_precision: 0.0291711
[15]	valid_0's l2: 0.0134672	valid_0's average_precision: 0.

In [8]:
importance = pd.DataFrame()
est_colname = ['importance0','importance1','importance2']
reg_colname = ['importance_0','importance_1','importance_2','importance_3','importance_4']
for i in range(5):
    for j in range(3):
        # "regs" is not defined unless you run the codes above!
        est = regs_defog[i].estimators_[j]
        name = est_colname[j]
        feature_importances = pd.DataFrame(est.feature_importances_,columns=[name])#.sort_values('importance')
        importance = pd.concat([importance,feature_importances],axis=1)
    name2 = reg_colname[i]
    importance[name2] = (importance['importance0'] + importance['importance1'] +importance['importance2']) / 3
    importance = importance.drop(columns=['importance0','importance1','importance2'])

importance['importance'] = (importance['importance_0']+importance['importance_1']+importance['importance_2']+importance['importance_3']+importance['importance_4']) / 5
importance = pd.DataFrame(importance['importance'],columns=['importance']).sort_values('importance')

for i in range(len(cols_defog)):
    importance = importance.rename({i:cols_defog[i]})

import plotly.express as px
fig = px.bar(x=importance.index.values,y=importance['importance'],color_discrete_sequence=['darkslateblue'])
fig.update_layout(xaxis_title="Feature", yaxis_title="Importance")
fig.show()

importance

,importance
AccAP_mean_3000,14.466667
rolling_avg_std_AP_500,14.600000
AccAP_mean_2000,15.466667
AccML_mean_3000,19.266667
rolling_avg_std_AP_2000,20.933333
Time_frac,25.266667


In [9]:
importance = pd.DataFrame()
est_colname = ['importance0','importance1','importance2']
reg_colname = ['importance_0','importance_1','importance_2','importance_3','importance_4']
for i in range(5):
    for j in range(3):
        # "regs" is not defined unless you run the codes above!
        est = regs_tdcsfog[i].estimators_[j]
        name = est_colname[j]
        feature_importances = pd.DataFrame(est.feature_importances_,columns=[name])#.sort_values('importance')
        importance = pd.concat([importance,feature_importances],axis=1)
    name2 = reg_colname[i]
    importance[name2] = (importance['importance0'] + importance['importance1'] +importance['importance2']) / 3
    importance = importance.drop(columns=['importance0','importance1','importance2'])

importance['importance'] = (importance['importance_0']+importance['importance_1']+importance['importance_2']+importance['importance_3']+importance['importance_4']) / 5
importance = pd.DataFrame(importance['importance'],columns=['importance']).sort_values('importance')

for i in range(len(cols_tdcsfog)):
    importance = importance.rename({i:cols_tdcsfog[i]})

import plotly.express as px
fig = px.bar(x=importance.index.values,y=importance['importance'],color_discrete_sequence=['darkslateblue'])
fig.update_layout(xaxis_title="Feature", yaxis_title="Importance")
fig.show()

importance

,importance
AccV_mean_3000,14.533333
rolling_avg_std_ML_2000,15.600000
rolling_avg_std_AP_2000,15.733333
rolling_avg_std_ML_1000,17.533333
AccV_mean_2000,18.400000
rolling_avg_std_AP_1000,18.800000
AccAP_mean_500,19.066667
AccAP_mean_1000,19.266667
rolling_avg_std_V_2000,19.400000
AccV_mean_5000,20.200000


## Predict for test

In [10]:
sub['t'] = 0
submission = []
for f in test:
    df = pd.read_csv(f)
    df.set_index('Time', drop=True, inplace=True)

    df['Id'] = f.split('/')[-1].split('.')[0]
    
    if 'defog' in str(f):
        df = preprocess_defog(df)
    else:
        df = preprocess_tdcsfog(df)
    
    # remove the columns from their original positions
    id_ = df.pop('Id')
    subject = df.pop('Subject')

    # add the columns at the end
    df.insert(0, 'Subject', subject)
    df.insert(0, 'Id', id_)
        
    # assume df is your DataFrame
    cols_to_scale = df.columns[2:]
    scaler = MinMaxScaler(feature_range=(-1, 1))

    df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])
    
    res_vals=[]
    
    for i_fold in range(N_FOLDS):
        if 'defog' in f:
            res_val=np.round(regs_defog[i_fold].predict(df[cols_defog]).clip(0.0,1.0),3)
            
        else:
            res_val=np.round(regs_tdcsfog[i_fold].predict(df[cols_tdcsfog]).clip(0.0,1.0),3)
        res_vals.append(np.expand_dims(res_val,axis=2))
    res_vals=np.mean(np.concatenate(res_vals,axis=2),axis=2)
    res = pd.DataFrame(res_vals, columns=pcols)
    
    df = pd.concat([df,res], axis=1)
    df['Id'] = df['Id'].astype(str) + '_' + df.index.astype(str)
    submission.append(df[scols])
    
submission = pd.concat(submission)
submission = pd.merge(sub[['Id']], submission, how='left', on='Id').fillna(0.0)
submission[scols].to_csv('submission.csv', index=False)
submission

,Id,StartHesitation,Turn,Walking
0,003f117e14_0,0.009,0.0522,0.0202
1,003f117e14_1,0.009,0.0522,0.0202
2,003f117e14_2,0.009,0.0522,0.0202
3,003f117e14_3,0.009,0.0522,0.0202
4,003f117e14_4,0.009,0.0522,0.0202
...,...,...,...,...
286365,02ab235146_281683,0.000,0.0214,0.0090
286366,02ab235146_281684,0.000,0.0214,0.0090
286367,02ab235146_281685,0.000,0.0214,0.0090
286368,02ab235146_281686,0.000,0.0214,0.0090
